In [0]:
from pyspark.sql.functions import col, explode, to_date, lit
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

patient_df = spark.read.format("delta").table("fhir_patient")
meds_df = spark.read.format("delta").table("fhir_medicationrequest")
cond_df = spark.read.format("delta").table("fhir_condition")
obs_df = spark.read.format("delta").table("fhir_observation")

In [0]:
person = patient_df.select(
    col("id").alias("person_id"),
    to_date("birthDate").alias("birth_datetime"),
    lit(0).alias("gender_concept_id")  # Map FHIR `gender` to OMOP concept_id via lookup if available
)

person.write.mode("append").format("delta").saveAsTable("person")

In [0]:
# Filter to antibiotics using RxNorm codes
abx_codes = ["723", "5640", "197361"]  # Amoxicillin, Penicillin, Azithromycin

drug_exposure = meds_df \
    .filter(col("medicationCodeableConcept.coding")[0].code.isin(abx_codes)) \
    .select(
        col("subject.reference").substr(9, 100).alias("person_id"),  # "Patient/123" → "123"
        to_date("authoredOn").alias("drug_exposure_start_date"),
        col("medicationCodeableConcept.coding")[0].code.alias("drug_concept_id")
    )
    
drug_exposure.write.mode("append").format("delta").saveAsTable("drug_exposure")

In [0]:
condition_occurrence = cond_df \
    .filter(col("code.coding")[0].code == "414916001") \
    .select(
        col("subject.reference").substr(9, 100).alias("person_id"),
        to_date("onsetDateTime").alias("condition_start_date"),
        lit("414916001").alias("condition_concept_id")
    )
condition_occurrence.write.mode("append").format("delta").saveAsTable("condition_occurrence")

In [0]:
measurement = obs_df \
    .filter(col("code.coding")[0].code == "39156-5") \
    .select(
        col("subject.reference").substr(9, 100).alias("person_id"),
        to_date("effectiveDateTime").alias("measurement_date"),
        col("valueQuantity.value").alias("value_as_number"),
        lit("3038553").alias("measurement_concept_id")  # BMI
    )
measurement.write.mode("append").format("delta").saveAsTable("measurement")